In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import PIL
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import random

from ConvBN import ConvBN as ConvBN
from LinearBN import LinearBN

In [2]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) # Normalize with mean 0.5 and std 0.5
])

batch_size= 1500
num_workers=2
pin_memory=True

g = torch.Generator()
g.manual_seed(42)

dataset = torchvision.datasets.MNIST(root='../Data', train=True, download=True, transform=transform)
train_set, val_set = torch.utils.data.random_split(dataset, [58000, 2000])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory, generator=g)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

test_set = torchvision.datasets.MNIST(root='../Data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)


In [4]:
if torch.cuda.is_available():
    print("CUDA is available")
else:
    print("CUDA is not available")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CUDA is available


In [5]:
class LReLU(nn.Module):
    def __init__(self):
        super(LReLU, self).__init__()
        self.alpha = nn.Parameter(torch.tensor(5.0)) 
    def forward(self, x):
        return torch.nn.functional.relu(self.alpha*x)

In [7]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        self.conv1_out = 32
        self.conv1_size = 3
        self.conv1_padding = 1


        self.conv2_out = 64
        self.conv2_size = 5
        self.conv2_padding = 2

        self.fc1_out = 512
        self.fc2_out = 10

        self.q = 1e-6
        self.bias_trick_par = nn.Parameter(torch.tensor(0.00005))

        # First Convolutional Block

        self.block1 = ConvBN(in_channels=1, out_channels=self.conv1_out, kernel_size=self.conv1_size, padding=self.conv1_padding, std = .05)
        self.block2 = ConvBN(in_channels=self.conv1_out, out_channels=self.conv2_out, kernel_size=self.conv2_size, padding=self.conv2_padding, std = .05)

        # Second Convolutional Block
       
        self.block3 = LinearBN(in_features = self.conv2_out * (28//2//2) * (28//2//2), out_features=self.fc1_out, std=.3)
        
        
        torch.manual_seed(0)
        self.w2 = nn.Parameter(torch.randn(self.fc1_out, self.fc2_out))
        nn.init.normal_(self.w2, mean=0.0, std=.6)

        self.dropout = nn.Dropout(0.5)

        self.relu = LReLU()




    def forward(self, x):
        
        x = F.max_pool2d(self.relu(self.block1(x)), (2,2), padding=0)
        x = F.max_pool2d(self.relu(self.block2(x)), (2,2), padding=0)
        
        x = x.view(x.size(0), -1)
        
        x = self.relu(self.block3(x))
        x = self.dropout(x)

        x = x + self.bias_trick_par
        x_norm = x / (x.norm(p=2, dim=1, keepdim=True) + self.q)  # Normalize input x
        w2_norm = self.w2 / (self.w2.norm(p=2, dim=1, keepdim=True) + self.q)  # Normalize weights
        x = torch.matmul(x_norm, w2_norm) # Matrix multiplication 

        # Return raw logits (no softmax here, CrossEntropyLoss handles it)
        return x

    def custom_round(self, n):
        remainder = n % 1000
        base = n - remainder
        if remainder >= 101:
            return base + 1000
        elif remainder <= 100:
            return base
            

    def init_hdc(self, ratio, seed):
        if not isinstance(ratio, (tuple)):
            raise TypeError("ratio must be a tuple of size 3")

        if not isinstance(seed, (tuple)):
            raise TypeError("seed must be a tuple of size 3")
        
        self.block1.init_hdc(ratio = ratio[0], seed = seed[0])
        self.block2.init_hdc(ratio = ratio[1], seed = seed[1])
        self.block3.init_hdc(ratio = ratio[2], seed = seed[2])
                
        n_last = self.w2.size(0)
        self.nHDC_last = int(self.custom_round(ratio[3] * n_last))
        torch.manual_seed(seed[3])
        self.g = torch.sign(torch.randn(self.w2.size(0), self.nHDC_last, device=self.w2.device)).to(torch.half)
        self.wg = torch.sign(torch.matmul(self.g.t(), self.w2.to(torch.half)))

    def hdc(self, x):
        x = F.max_pool2d(self.relu(self.block1.hdc(x)), (2,2), padding=0)
        x = F.max_pool2d(self.relu(self.block2.hdc(x)), (2,2), padding=0)

        x = x.view(x.size(0), -1)
        x = self.relu(self.block3.hdc(x))

        x = x + self.bias_trick_par
        x = torch.sign(torch.matmul(x.to(torch.half), self.g))

        return x
        
    def classification_layer(self, x):
        x = x @ self.wg
        return x


In [8]:
from tqdm import tqdm
import time
from torch.nn.parallel import data_parallel
from torch.utils.data import Subset


torch.cuda.empty_cache()
model = Network().to(device)
model.load_state_dict(torch.load('MNIST_GNet_Training_99.30.pth', weights_only = True))



model.to(torch.half).to(device)
model.eval()

n_splits = 20
split_size = len(test_set) // n_splits 

accuracies = np.zeros((11, n_splits))
# scale = range
scales = np.arange(1.5, 1.55, 0.1)
hyperdims = np.zeros((len(scales), 4))
for i, scale in enumerate(scales):
    indices = list(range(len(test_set)))
    # np.random.seed(42)
    # np.random.shuffle(indices)
    for split_idx in tqdm(range(n_splits)):
        start_idx = split_idx * split_size
        end_idx = start_idx + split_size
        split_indices = indices[start_idx:end_idx]
        split_subset = Subset(test_set, split_indices)
        split_loader = torch.utils.data.DataLoader(split_subset, batch_size=3, shuffle=False,
                                                   num_workers=num_workers, pin_memory=pin_memory)
        ratio = (12, 1.15/6, 3, 18)
        ratio = tuple(scale * r for r in ratio)
        torch.manual_seed(split_idx+15)
        random_seeds = tuple(torch.randint(0, 1000, (1,)).item() for _ in range(4))
        torch.cuda.empty_cache()
        
        model.init_hdc(ratio, random_seeds)
        hyperdims[i] = np.array([model.block1.nHDC, model.block2.nHDC, model.block3.nHDC, model.nHDC_last])
        correct = 0
        total = 0
    
        with torch.no_grad():
            for images, labels in (split_loader):
                images, labels = images.cuda(non_blocking=True), labels.cuda(non_blocking=True)
                output = model.hdc(images.to(torch.half))
                output = model.classification_layer(output.to(torch.half))
                _, predicted = torch.max(output.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
        acc = 100 * correct / total
    
        accuracies[i, split_idx] = acc
    
    print(f'Block1: {model.block1.nHDC}, Block2: {model.block2.nHDC}, Block3: {model.block3.nHDC} Classification Layer: {model.nHDC_last}, Average Accuracy: {np.mean(accuracies[i]):.2f}%')


100%|██████████| 20/20 [03:33<00:00, 10.69s/it]

Block1: 6000, Block2: 15000, Block3: 15000 Classification Layer: 14000, Average Accuracy: 99.03%
